In [1]:
import pandas as pd
import sys
import os

# Add src to path for imports
sys.path.append(os.path.join(os.getcwd(), 'src'))
from format_persona_data import process_raw_data, generate_persona_json

ModuleNotFoundError: No module named 'format_persona_data'

In [ ]:
participant_data_fname = "src/GD1_participants.csv"

In [ ]:
(enligsh_only, raw_langauge) = process_raw_data(participant_data_fname)

In [ ]:
# Save an example persona

generate_persona_json(enligsh_only[20])

In [ ]:
questions_to_consider =  [
    'What is another dangerous outcome you can imagine happening from AI? What would need to be true for that to happen?',
    'What is another positive outcome you can imagine happening from AI? What would need to be true for that to happen?',
    'What are the first decisions in your life that you would delegate to an AI?',
    'Describe a concrete scenario where you would feel uncomfortable with an AI making a decision instead of a human. What specific aspects make you uneasy?',
    'What is ONE specific ethical or moral principle, virtue, or value that you think is core to how you live your life? Please explain it in detail.',
    'What is one ethical or moral principle, virtue, or value that you think is important but not reflected in the way technology is developed today? Please explain it in detail.',
    'What do you consider your culture? What is your favorite thing about it?',
    'What is one unique perspective that your culture brings that you want to preserve into the future?',
    'What kinds of cultural things would you be most worried about losing in a future with advanced AI?',
]

## Running a session with the persona

You can now run red teaming sessions either directly (as shown below) or via the API.

In [ ]:
import yaml
import os
from string import Template
import sys

# Add src to path for agents import
sys.path.append(os.path.join(os.getcwd(), 'src'))
from agents.base_types import Persona, Conversation, ConversationTurn

verbose = True
goal_generator_fname = os.path.join('src', 'configs', 'simulacra_goal_generator.yaml')
target_agent_config = os.path.join('src', 'configs', 'nora.yaml')
persona_fname = os.path.join('src', 'baseline_personas','2d33bd32-ca14-4ab2-813e-8b1eb0fea04d_English.json')

In [ ]:
# from agents.run_session import run_session_from_config
# output = await run_session_from_config(
#     persona_config=persona_fname,
#     target_agent_config=target_agent_config,
#     verbose=verbose,
#     # num_goals=5,
#     # max_turns=10,
# )

In [ ]:
# output['good_faith']

## Using the API

Alternatively, you can use the API to run red teaming sessions. Start the API server first:

```bash
python app/api/run_api.py
```

The API will be available at `http://localhost:8000`. You can check the API documentation at `app/api/README.md`.

Then use it programmatically:

In [ ]:
# Example using requests library to interact with the API
import requests
import json

# API base URL
API_BASE = "http://localhost:8000"

def check_api_health():
    """Check if the API server is running"""
    try:
        response = requests.get(f"{API_BASE}/health")
        if response.status_code == 200:
            print(f"✅ API Status: {response.json()['status']}")
            return True
        else:
            print(f"❌ API returned status code: {response.status_code}")
            return False
    except requests.exceptions.ConnectionError:
        print("❌ API server is not running. Start it with: python app/api/run_api.py")
        return False
    except Exception as e:
        print(f"❌ Error connecting to API: {e}")
        return False

def run_session_via_api(persona_id, num_goals=5, max_turns=10, use_db=True):
    """Run a red teaming session via the API"""
    session_data = {
        "persona_fname": persona_id,  # Use persona ID for database lookup
        "num_goals": num_goals,
        "max_turns": max_turns,
        "use_db": use_db,
        "verbose": True
    }
    
    try:
        response = requests.post(f"{API_BASE}/run-red-teaming-session", json=session_data)
        
        if response.status_code == 200:
            result = response.json()
            if result.get("success"):
                print(f"✅ Session completed successfully")
                print(f"Session data keys: {list(result.get('session_data', {}).keys())}")
                return result
            else:
                print(f"❌ Session failed: {result.get('error', 'Unknown error')}")
                return None
        else:
            print(f"❌ API request failed with status code: {response.status_code}")
            print(f"Response: {response.text}")
            return None
            
    except Exception as e:
        print(f"❌ Error making API request: {e}")
        return None

def list_personas():
    """List available personas in the database"""
    try:
        response = requests.get(f"{API_BASE}/personas")
        if response.status_code == 200:
            personas = response.json()
            print(f"Found {len(personas)} personas:")
            for persona in personas[:5]:  # Show first 5
                print(f"  - {persona['id']} ({persona['response_language']})")
            if len(personas) > 5:
                print(f"  ... and {len(personas) - 5} more")
            return personas
        else:
            print(f"❌ Failed to load personas: HTTP {response.status_code}")
            return []
    except Exception as e:
        print(f"❌ Error loading personas: {e}")
        return []

# Test the API
print("Testing API connection...")
if check_api_health():
    print("\n📋 Available personas:")
    personas = list_personas()
    
    if personas:
        print(f"\n🎯 Running example session with database persona...")
        # Use persona ID (database mode) instead of file path
        example_persona_id = "2d33bd32-ca14-4ab2-813e-8b1eb0fea04d_English"
        
        result = run_session_via_api(example_persona_id, num_goals=2, max_turns=3, use_db=True)
        
        if result:
            print(f"\n✅ Session completed! Check the results above.")
    
    print(f"\n🌐 Streamlit Frontend Available!")
    print(f"For a better user experience, use the Streamlit frontend:")
    print(f"1. Run: python app/frontend/run_streamlit.py")
    print(f"2. Open: http://localhost:8501")
    print(f"3. Interactive interface with persona browser and session management")
    
else:
    print("Please start the API server first: python app/api/run_api.py")

Testing API connection...
✅ API Status: healthy

Running example session...
❌ API request failed with status code: 404
Response: {"detail":"Not Found"}


## 🌐 Streamlit Frontend

For a more user-friendly experience, we've created a Streamlit web interface that provides:

### Features
- **👥 Interactive Persona Browser**: Browse and select personas visually
- **🎯 Session Configuration**: Easy-to-use forms for configuring red teaming sessions  
- **📊 Results Visualization**: Formatted display of conversation results
- **💾 Export Functionality**: Download session results as JSON files

### Quick Start

1. **Ensure API is running**:
   ```bash
   python app/api/run_api.py
   ```

2. **Start Streamlit frontend**:
   ```bash
   python app/frontend/run_streamlit.py
   ```

3. **Open in browser**: http://localhost:8501

### Screenshots
The Streamlit app provides three main pages:
- **Browse Personas**: Grid view of all available personas with selection
- **Run Session**: Configure and execute red teaming sessions
- **Session Results**: View formatted conversation results with export options

The interface is much more intuitive than the API calls below, especially for non-technical users!